# Extract src from question text

In [1]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [2]:
import preprocess as pp
import pandas as pd
import numpy as np
from constants import CONSTANTS
import pickle

In [ ]:
input_year = '2023'
input_pl = 'python'
year_range = '2223'

In [4]:
# 파일 불러오기
with open(f'{CONSTANTS.data_path}/data/complexity_{input_pl}_df_{input_year}_{year_range}.pkl', 'rb') as f:
    complexity_python_df = pickle.load(f)

AttributeError: type object 'CONSTANTS' has no attribute 'data_path'

In [ ]:
complexity_python_df['cdate'].max()

In [ ]:
complexity_python_df['cdate'].max()

In [ ]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()

In [ ]:
complexity_python_df['c_h_body'] = complexity_python_df['body'].apply(lambda x : htmlp.clean_html_str(x))

In [ ]:
complexity_python_df.info()

In [ ]:
complexity_python_df['c_body'] = complexity_python_df['body'].apply(lambda x : codep(x))

In [ ]:
complexity_python_df

In [ ]:
complexity_python_src = complexity_python_df[['id', 'c_body']]

In [ ]:
complexity_python_src['code_len'] = complexity_python_src['c_body'].apply(lambda x: len(x['code_sections']))

In [ ]:
complexity_python_src = complexity_python_src[complexity_python_src['code_len']>0]

In [ ]:
complexity_python_src.head(5)

In [ ]:
src_df = pd.DataFrame(columns = ['id', 'src'])
def code_explode(row) : 
    global src_df
    row['id']
    src_arr = []
    for i in range(row['code_len']):
        tmp_src_arr = []
        tmp_src_arr.append(row['id'])
        tmp_src_arr.append(row['c_body']['code_sections'][i]['span_str'])
        src_df = src_df.append(pd.Series(tmp_src_arr, index=src_df.columns), ignore_index=True)
    return True
        

In [ ]:
# bert_src_ex = bert_src[['q_id', 'q_prep_text']].apply(pd.Series.explode)
complexity_python_src.apply(code_explode, axis = 1)

In [ ]:
src_df.reset_index(inplace=True)

In [ ]:
src_df.head()

In [ ]:
for i in range(src_df.shape[0]):
    idx = src_df.iloc[i, 0]
    qid = src_df.iloc[i, 1]
    src = src_df.iloc[i, 2]
    file_nm = str(idx)+'_' + str(qid)+'.py'
    with open(f'../../data/src/{input_year}_src/{file_nm}', 'wb') as f:
        f.write(src.encode())
